In [17]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import math
import re
from collections import Counter
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import smart_open
import json

In [18]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')
movies

budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4797     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4798       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4799          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4800          0                                                 []   
4801          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.go.com/disneypictures/pirates/     285   
2           http://www.sonypictures.com/movies/spectre/  206647   
3                    http://www.thedarkknightrises.com/   49026   
4                  http://movies.disney.com/john-carter   49529   
...                                                 ...     ...   
4797                                                NaN    9367   
4798                                                NaN   72766   
4799  http://www.hallmarkchannel.com/signedsealeddel...  231617   
4800                        http://shanghaicalling.com/  126186   
4801                                                NaN   25975   

                                               keywords original_language  \
0     [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1     [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2     [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3     [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4     [{"id": 818, "name": "based on novel"}, {"id":...                en   
...                                                 ...               ...   
4797  [{"id": 5616, "name": "united states\u2013mexi...                es   
4798                                                 []                en   
4799  [{"id": 248, "name": "date"}, {"id": 699, "nam...                en   
4800                                                 []                en   
4801  [{"id": 1523, "name": "obsession"}, {"id": 224...                en   

                                original_title  \
0                                       Avatar   
1     Pirates of the Caribbean: At World's End   
2                                      Spectre   
3                        The Dark Knight Rises   
4                                  John Carter   
...                                        ...   
4797                               El Mariachi   
4798                                 Newlyweds   
4799                 Signed, Sealed, Delivered   
4800                          Shanghai Calling   
4801                         My Date with Drew   

                                               overview  popularity  \
0     In the 22nd century, a paraplegic Marine is di...  150.437577   
1     Captain Barbossa, long believed to be dead, ha...  139.082615   
2     A cryptic message from Bond’s past sends him o...  107.376788   
3     Following the death of District Attorney Harve...   112.31295   
4     John Carter is a war-weary, former military ca...   43.926995   
...                                                 ...         ...   
4797  El Mariachi just wants to play his guitar and ...   14.269792   
4798  A newlywed couple's honeymoon is upended by th...    0.642552   
4799  "Signed, Sealed, Delivered" introduces a dedic...    1.444476   
4800  When ambitious New York attorney Sam is sent t...    0.857008   
4801  Ever since the second grade when he first saw ... 

In [19]:
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')


In [20]:
movies_clean=movies_df_merged[['id','original_title','overview']]
movies_clean

id                            original_title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4797    9367                               El Mariachi   
4798   72766                                 Newlyweds   
4799  231617                 Signed, Sealed, Delivered   
4800  126186                          Shanghai Calling   
4801   25975                         My Date with Drew   

                                               overview  
0     In the 22nd century, a paraplegic Marine is di...  
1     Captain Barbossa, long believed to be dead, ha...  
2     A cryptic message from Bond’s past sends him o...  
3     Following the death of District Attorney Harve...  
4     John Carter is a war-weary, former military ca...  
...                                                 ...  
4797  El Mariachi just wants to play his guitar and ...  
4798  A newlywed couple's honeymoon is upended by th...  
4799  "Signed, Sealed, Delivered" introduces a dedic...  
4800  When ambitious New York attorney Sam is sent t...  
4801  Ever since the second grade when he first saw ...  

[4802 rows x 3 columns]

In [21]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)


In [22]:
stopwords = []
corpus=[]
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,4802):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    corpus.append(text_words)

In [23]:
def read_corpus(corpus):
    for i, line in enumerate(corpus):
        yield gensim.models.doc2vec.TaggedDocument(line, [i])
train_corpus = list(read_corpus(corpus))

In [24]:
#PV-DBOW
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
movie_vector=[]
for i in range(0,4802):
    movie_vector.append(model.infer_vector(corpus[i]))
movie_vector

[array([-0.39152443, -0.26833707,  0.17711039,  0.06678433,  0.4198531 ,
        -0.6398309 , -0.28078735,  0.31926835, -0.00403576, -0.26345786,
        -0.04085909,  0.39448223,  0.40547836, -0.14533724,  0.3223869 ,
        -0.16708772,  0.14586782,  0.01783369, -0.28111577,  0.23502852,
         0.02932054,  0.57561743, -0.18799625,  0.44648746, -0.5947958 ,
         0.3751866 ,  0.09600512, -0.05854001, -0.12086388,  0.2833729 ],
       dtype=float32),
 array([ 0.25158152,  0.10391496, -0.16499816,  0.70528394,  0.361665  ,
        -0.43714014, -0.3794247 ,  0.10598366,  0.00750517, -0.2770184 ,
        -0.32686016, -0.07253183, -0.34169403, -0.35478416,  0.01201697,
        -0.29151502, -0.1289758 ,  0.09271314,  0.61696994,  0.22256428,
        -0.21195197,  0.4283258 , -0.09070865,  1.2360092 , -0.44411823,
         0.44957617, -0.06651967,  0.44451082, -0.6291407 , -0.15913704],
       dtype=float32),
 array([-0.8632849 , -0.4621836 ,  0.6458882 , -0.36289605, -0.12618826,
   

In [25]:
#PV-DM
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=2, epochs=40,dm=1)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)
movie_vector=[]
for i in range(0,4802):
    movie_vector.append(model.infer_vector(corpus[i]))
movie_vector

[array([-0.17039856, -0.47612113, -0.09208493, -0.04980542,  0.03148771,
        -0.5592856 ,  0.02222181,  0.21053849, -0.23538537,  0.06514801,
         0.02118761,  0.3124067 ,  0.4236693 , -0.22763814,  0.41501763,
        -0.15141617,  0.14134118, -0.18677944, -0.3244956 ,  0.26126468,
        -0.05554755,  0.45596257, -0.12597641,  0.29528657, -0.72126395,
         0.40744516,  0.08308642, -0.2268463 , -0.2304043 ,  0.27637404],
       dtype=float32),
 array([ 0.08952631, -0.01444266, -0.18706396,  0.7261775 , -0.05330686,
        -0.40206456, -0.06048949, -0.14252226,  0.38773218, -0.03943819,
        -0.11683258, -0.3360831 , -0.11245248,  0.09208579,  0.5451756 ,
        -0.08864187,  0.06153368,  0.11995251,  0.46048713,  0.40596387,
        -0.7237087 ,  0.36475992, -0.14716272,  1.2158649 ,  0.00252751,
         0.14055178,  0.10915768, -0.08106732, -0.7965091 , -0.18786468],
       dtype=float32),
 array([-0.8147001 , -0.46739373,  0.56365174, -0.35956   , -0.37981665,
   

In [26]:
movie_vector=[]
for i in range(0,4802):
    movie_vector.append(model.infer_vector(corpus[i]))


In [27]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)

In [28]:
def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

In [29]:
def recommend(movie):
    vector=model.infer_vector(corpus[indices[movie]])
    sims = model.docvecs.most_similar([vector],topn=10)
    movies_indices=[]
    for i in sims:
        movies_indices.append(i[0])
    print("Movie related to %s"%(movie))
    return movies_clean.original_title.iloc[movies_indices]

In [30]:
recommend('Avatar')

Movie related to Avatar


C:\Users\ASUS\AppData\Local\Temp\ipykernel_17556\691199421.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([vector],topn=10)


0                       Avatar
1914                 Lifeforce
45                 World War Z
4542          The Californians
653             This Means War
1091                     Nixon
4720    The Work and The Story
2327                  Predator
3068               Rescue Dawn
3021           Invasion U.S.A.
Name: original_title, dtype: object